In [1]:
from urllib.request import urlopen

data = []
url = "https://raw.githubusercontent.com/PantheonPlusSH0ES/DataRelease/main/Pantheon%2B_Data/4_DISTANCES_AND_COVAR/Pantheon%2BSH0ES.dat"
for line in urlopen(url):
    vals = line.split(b" ")
    
    # skip the headers
    if vals[0] == b"CID": continue 
    
    data.append({"z": float(vals[2]), "mu": float(vals[10])})

In [2]:
import cosmolopy.magnitudes as cm

# calculates Lambda-CDM
cosmo = {'omega_M_0' : 0.3, 'omega_lambda_0' : 0.7, 
         'h' : 0.70, 'omega_k_0': 0}

def calculateMu(z):
    return cm.distance_modulus(z, **cosmo)

In [3]:
def sse():
    totalErrors = 0
    for sn in data:
        if sn["z"] > 1: break
        
        error = calculateMu(sn["z"]) - sn["mu"]
        totalErrors = totalErrors + error**2

    return totalErrors

In [4]:
lowestSSE = {"sse": -1}
for i in range(30):
    H = 69 + i/5
    cosmo["h"] = H/100
    
    SSE = sse()
    if lowestSSE["sse"] == -1 or SSE < lowestSSE["sse"]:
        lowestSSE = {"H": H, "sse": SSE}

print(lowestSSE)

{'H': 73.2, 'sse': 49.776740614452095}


In [5]:
import math

def calculateMu(z):
    d_L = 299792/H * (1+z) * math.log(1+z)
    mu = 5 * math.log10(d_L) + 25
    return mu